In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


/content/drive/MyDrive/SuperAI/Project/GDA/images

# **Training**

# Convert YOLO 1.1 format to YOLO-OBB format

In [8]:
import os

def convert_aabb_to_obb(aabb_labels_dir, obb_labels_dir):
    # Ensure the OBB directory exists
    os.makedirs(obb_labels_dir, exist_ok=True)

    for label_file in os.listdir(aabb_labels_dir):
        aabb_path = os.path.join(aabb_labels_dir, label_file)
        obb_path = os.path.join(obb_labels_dir, label_file)

        with open(aabb_path, 'r') as f:
            lines = f.readlines()

        with open(obb_path, 'w') as f:
            for line in lines:
                class_id, x_center, y_center, width, height = map(float, line.split())
                # Convert center, width, height to x1, y1, x2, y2, x3, y3, x4, y4
                # Assuming no rotation, the OBB is the same as the AABB
                x1 = x_center - width / 2
                y1 = y_center - height / 2
                x2 = x_center + width / 2
                y2 = y_center - height / 2
                x3 = x_center + width / 2
                y3 = y_center + height / 2
                x4 = x_center - width / 2
                y4 = y_center + height / 2
                # Write the converted annotation to the OBB label file with spaces
                f.write(f"{int(class_id)} {x1} {y1} {x2} {y2} {x3} {y3} {x4} {y4}\n")

# Define your directories
aabb_labels_dir = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_labels'
obb_labels_dir = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels'

convert_aabb_to_obb(aabb_labels_dir, obb_labels_dir)


# Split dataset to train and val

In [9]:
import os
import shutil
from random import shuffle
from sklearn.model_selection import train_test_split

def find_files(directory, extension=".jpg"):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(extension)]

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def copy_files(pairs, directory, label_directory):
    # Ensure target directories exist
    ensure_dir(directory)
    ensure_dir(label_directory)

    for img_path, label_path in pairs:
        img_name = os.path.basename(img_path)
        label_name = os.path.basename(label_path)

        shutil.copy2(img_path, os.path.join(directory, img_name))
        shutil.copy2(label_path, os.path.join(label_directory, label_name))
        print(f"Copied {img_path} to {os.path.join(directory, img_name)}")
        print(f"Copied {label_path} to {os.path.join(label_directory, label_name)}")

def split_data(images, labels, train_dir, val_dir, train_labels_dir, val_labels_dir, val_ratio=0.2):
    paired_data = list(zip(images, labels))
    train_pairs, val_pairs = train_test_split(paired_data, test_size=val_ratio, random_state=42)

    # Copy the files to train and validation directories
    copy_files(train_pairs, train_dir, train_labels_dir)
    copy_files(val_pairs, val_dir, val_labels_dir)

# Your directory paths
source_images_path = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images'
source_labels_path = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels'
train_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train' #'/content/drive/MyDrive/SuperAI/Project/GDA/train/images'
val_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/val' #'/content/drive/MyDrive/SuperAI/Project/GDA/validation/images'
train_labels_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train' #'/content/drive/MyDrive/SuperAI/Project/GDA/train/labels'
val_labels_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/val' #'/content/drive/MyDrive/SuperAI/Project/GDA/validation/labels'

# Find all image and label files
image_files = find_files(source_images_path)
label_files = find_files(source_labels_path, extension=".txt")

# Split and copy the images and labels to train and validation directories
split_data(image_files, label_files, train_directory_path, val_directory_path,
           train_labels_directory_path, val_labels_directory_path, val_ratio=0.2)


Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0084.jpg to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0084.jpg
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels/0021.txt to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train/0021.txt
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0043.jpg to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0043.jpg
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels/0011.txt to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train/0011.txt
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0002.jpg to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0002.jpg
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels/0118.txt to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train/0118.txt
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0010.

# write YAML file

In [11]:
# Content to be written to the YAML file
yaml_content = """# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB # dataset root dir
train: /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train # train images (relative to 'path')
val: /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/val # val images (relative to 'path')
#test: /content/drive/MyDrive/SuperAI/Project/GDA/test # test images (optional)

# Classes
names:
  0: Energy
  1: Sugar
  2: Fat
  3: Sodium
"""

# Write to the YAML file
with open('/content/drive/MyDrive/SuperAI/Project/GDA/GDA_yolo-obb.yaml', 'w') as file:
    file.write(yaml_content)

print('GDA_yolo-obb.yaml has been created.')


GDA_yolo-obb.yaml has been created.


# YOLO-OBB

In [12]:
# Install the ultralytics package from PyPI
!pip install ultralytics

In [13]:
from ultralytics import YOLO

# Create a new YOLOv8n-OBB model from scratch
model = YOLO('yolov8n-obb.yaml')

# # Train the model on the GDA_yolo-obb.yaml dataset
results = model.train(data='/content/drive/MyDrive/SuperAI/Project/GDA/GDA_yolo-obb.yaml', epochs=200, imgsz=1600) #epochs=100, imgsz=640)

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.yaml, data=/content/drive/MyDrive/SuperAI/Project/GDA/GDA_yolo-obb.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=1600, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 755k/755k [00:00<00:00, 122MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 70.1MB/s]


AMP: checks passed ✅


...

    200/200      13.7G     0.5263      1.159      2.345         24       1600: 100%|██████████| 5/5 [00:04<00:00,  1.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:08<00:00,  4.04s/it]

                   all         18         12      0.142       0.61      0.207      0.189



200 epochs completed in 1.440 hours.
Optimizer stripped from runs/obb/train/weights/last.pt, 6.6MB
Optimizer stripped from runs/obb/train/weights/best.pt, 6.6MB

Validating runs/obb/train/weights/best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-obb summary (fused): 187 layers, 3077999 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.28s/it]


                   all         18         12      0.205       0.75      0.287      0.257
                Energy         18          3      0.191      0.667      0.446      0.391
                 Sugar         18          3      0.143      0.667      0.143      0.131
                   Fat         18          3      0.224          1       0.23      0.212
                Sodium         18          3      0.262      0.667      0.328      0.295
Speed: 2.0ms preprocess, 19.0ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/obb/train


---

In [15]:
!cp -r /content/runs/obb/train /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/train

๒

---

# **Testing**

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [22]:
from ultralytics import YOLO

# Load the trained model
model = YOLO('/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/14032024/train/weights/best.pt')


In [30]:
# Perform inference on an image or a directory of images
# results = model.predict('/content/drive/MyDrive/SuperAI/Project/GDA/test')
results = model.predict('/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train')


image 1/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0001.jpg: 1600x1216 25.6ms
image 2/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0002.jpg: 1600x1216 25.3ms
image 3/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0004.jpg: 1600x1216 25.3ms
image 4/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0005.jpg: 1600x1216 25.3ms
image 5/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0007.jpg: 1216x1600 25.4ms
image 6/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0008.jpg: 1600x1216 26.5ms
image 7/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0009.jpg: 1600x1216 25.4ms
image 8/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0010.jpg: 1216x1600 99.8ms
image 9/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0011.jpg: 1216x1600 24.6ms
image 10/72 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0015.

# Crops and draw OBB

In [ ]:
import cv2
import numpy as np
import os

def save_crops_and_draw_obb(result, save_dir, crops_dir):
    """
    Saves crops for each label based on the axis-aligned bounding boxes from the OBB results and draws OBBs on the original image.

    Parameters:
    - result: A 'Results' object from Ultralytics YOLO inference containing predictions.
    - save_dir: Directory where the annotated images will be saved.
    - crops_dir: Directory where the cropped areas will be saved.
    """
    os.makedirs(save_dir, exist_ok=True)  # Ensure the save directory exists
    os.makedirs(crops_dir, exist_ok=True)  # Ensure the crops directory exists

    # Define colors for each label in BGR format
    colors = {0: (0, 0, 255),    # Red
              1: (147, 20, 255), # Pink
              2: (0, 165, 255),  # Orange
              3: (0, 255, 255)}  # Yellow

    # Use the original image for cropping to avoid drawing on it
    orig_image_for_cropping = result.orig_img.copy()
    # Use another copy for drawing to save with OBBs drawn on it
    image_for_drawing = result.orig_img.copy()

    if result.obb.xyxyxyxy.numel() > 0:
        obbs = result.obb.xyxyxyxy.cpu().numpy()  # OBB coordinates
        aabbs = result.obb.xyxy.cpu().numpy()  # Axis-aligned bounding boxes for cropping
        classes = result.obb.cls.cpu().numpy()  # Class IDs for each OBB

        original_basename = os.path.splitext(os.path.basename(result.path))[0]  # Remove file extension from original basename

        for i, (obb, aabb, cls_id) in enumerate(zip(obbs, aabbs, classes)):
            # Crop using axis-aligned bounding box from the original image
            x1, y1, x2, y2 = aabb.astype(int)
            crop = orig_image_for_cropping[y1:y2, x1:x2]
            crop_path = os.path.join(crops_dir, f"{original_basename}_class_{int(cls_id)}_crop_{i}.jpg")
            cv2.imwrite(crop_path, crop)

            # Draw OBB on the drawing image
            color = colors.get(int(cls_id), (255, 255, 255))  # Default to white if class ID is not in colors
            print(f"Drawing box for class {cls_id} with color {color}")  # Debug print
            points = obb.reshape((-1, 1, 2)).astype(np.int32)
            cv2.polylines(image_for_drawing, [points], isClosed=True, color=color, thickness=10)

    # Save the annotated original image with OBBs drawn
    save_path = os.path.join(save_dir, os.path.basename(result.path))
    cv2.imwrite(save_path, image_for_drawing)

save_dir = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/14032024/train-obb_results'
crops_dir = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/14032024/train-crops'

for result in results:
    save_crops_and_draw_obb(result, save_dir, crops_dir)


Drawing box for class 0.0 with color (0, 0, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 2.0 with color (0, 165, 255)
Drawing box for class 3.0 with color (0, 255, 255)
Drawing box for class 2.0 with color (0, 165, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 3.0 with color (0, 255, 255)
Drawing box for class 0.0 with color (0, 0, 255)
Drawing box for class 2.0 with color (0, 165, 255)
Drawing box for class 0.0 with color (0, 0, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 2.0 with color (0, 165, 255)
Drawing box for class 3.0 with color (0, 255, 255)
Drawing box for class 0.0 with color (0, 0, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 1.0 with color (147, 20, 255)
Drawing box for class 3.0 with color (0, 255, 255)
Drawing box for class 0.0 with c

------

# OCR part

In [ ]:
! pip install -q pytesseract
! apt install -qq tesseract-ocr tesseract-ocr-tha

In [ ]:
import cv2
import numpy as np

# Load the template GDA image (the image of the GDA label you provided)
template = cv2.imread(image_files[0], 0)
w, h = template.shape[::-1]

# Load the target image where you want to find the template
target_image_path = 'path_to_target_image.png'
img_rgb = cv2.imread(target_image_path)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

# Apply template Matching
res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
threshold = 0.8  # You might need to adjust this threshold
loc = np.where(res >= threshold)

# If the template is found in the image
if loc[0].size > 0:
    pt = loc[::-1][0][0], loc[::-1][1][0]
    # Draw a rectangle around the matched region
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 255, 255), 2)

    # Crop the matched region from the target image
    roi = img_rgb[pt[1]:pt[1] + h, pt[0]:pt[0] + w]

    # Save or process the ROI...

# Show the result
cv2.imshow('Detected', img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import pytesseract
from pytesseract import Output

# Load the image
image = cv2.imread(image_files[0])

# Preprocess the image (if necessary)
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding
thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Use pytesseract to do OCR on the processed image
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, config=custom_config, lang='tha+eng')

# Print the text
print(text)
